<a href="https://colab.research.google.com/github/TwistedGrims/sdfsdf32423dsfsdzxvczxfweqr2314/blob/main/imagegenerator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Check GPU
import torch
if not torch.cuda.is_available():
    print("⚠️ GPU not enabled. Go to Runtime > Change runtime type > GPU")

# Install dependencies
!pip install --upgrade diffusers transformers accelerate ftfy scipy

# Import pipeline
from diffusers import StableDiffusionXLPipeline
from PIL import Image

# Set device
device = "cuda" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if device == "cuda" else torch.float32

# Load model
model_name = "TheReprinter/WAI-NSFW-illustrious-SDXL"
pipe = StableDiffusionXLPipeline.from_pretrained(model_name, torch_dtype=torch_dtype)
pipe = pipe.to(device)


⚠️ GPU not enabled. Go to Runtime > Change runtime type > GPU
ERROR: Operation cancelled by user
